<a href="https://colab.research.google.com/github/arockiasachin/ContextualFinAi/blob/main/Unstructured_Chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%pip install -Uq "unstructured[all-docs]" pillow lxml pillow
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai langchain-groq
%pip install -Uq python_dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 29.7 MB/s e

In [ ]:
pip install unstructured-ingest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.0 MB/s eta 0:00:00


In [ ]:
import os

# keys for the services we will use

os.environ["OPENAI_API_KEY"] = "OPENAI API KEY HERE"
os.environ["GROQ_API_KEY"] = "GROQ API KEY HERE"
os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN API KEY HERE"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["UNSTRUCTURED_API_KEY"] = "UNSTRUCTURED API KEY HERE"
os.environ["UNSTRUCTURED_API_URL"] = "https://api.unstructuredapp.io/general/v0/general"
os.environ["LOCAL_FILE_INPUT_DIR"] = "INPUT FOLDER HERE"
os.environ["LOCAL_FILE_OUTPUT_DIR"] = "OUTPUT FOLDER HERE"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Unstructured Pipeline

In [1]:
import os


In [ ]:
import os
from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
    LocalIndexerConfig,
    LocalDownloaderConfig,
    LocalConnectionConfig,
    LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig
from unstructured_ingest.v2.processes.chunker import ChunkerConfig
from unstructured_ingest.v2.processes.embedder import EmbedderConfig

In [ ]:
Pipeline.from_configs(
    context=ProcessorConfig(),
    indexer_config=LocalIndexerConfig(input_path=os.getenv("LOCAL_FILE_INPUT_DIR")),
    downloader_config=LocalDownloaderConfig(),
    source_connection_config=LocalConnectionConfig(),
    partitioner_config=PartitionerConfig(
        partition_by_api=True,
        api_key=os.getenv("UNSTRUCTURED_API_KEY"),
        partition_endpoint=os.getenv("UNSTRUCTURED_API_URL"),
        strategy="hi_res",
        additional_partition_args={
            "split_pdf_page": True,
            "split_pdf_allow_failed": True,
            "split_pdf_concurrency_level": 15,
            "extract_image_block_types": ["Table"]
        }
    ),
    chunker_config=ChunkerConfig(
        chunking_strategy="by_title",
        chunk_max_characters=2000,
        chunk_new_after_n_chars=1500,
        chunk_combine_text_under_n_chars=400,
        chunk_multipage_sections=True,
        chunk_overlap=100,
        chunk_include_orig_elements=False,
        similarity_threshold=0.7,
        chunk_by_api=False  # Disable API-based chunking to use local chunking
    ),
    uploader_config=LocalUploaderConfig(output_dir=os.getenv("LOCAL_FILE_OUTPUT_DIR"))
).run()

Overriding of current TracerProvider is not allowed
2024-12-18 18:50:56,460 MainProcess INFO     created index with configs: {"input_path": "/content/drive/MyDrive/Final52RAG/input", "recursive": false}, connection configs: {"access_config": "**********"}
2024-12-18 18:50:56,463 MainProcess INFO     Created download with configs: {"download_dir": null}, connection configs: {"access_config": "**********"}
2024-12-18 18:50:56,465 MainProcess INFO     created partition with configs: {"strategy": "hi_res", "ocr_languages": null, "encoding": null, "additional_partition_args": {"split_pdf_page": true, "split_pdf_allow_failed": true, "split_pdf_concurrency_level": 15, "extract_image_block_types": ["Table"]}, "skip_infer_table_types": null, "fields_include": ["element_id", "text", "type", "metadata", "embeddings"], "flatten_metadata": false, "metadata_exclude": [], "element_exclude": [], "metadata_include": [], "partition_endpoint": "https://api.unstructuredapp.io/general/v0/general", "partiti

# Tables Extraction

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the model
model = ChatOpenAI(temperature=0.5, model="gpt-4o-mini")

# 1. Prompt for summarization
summary_prompt = ChatPromptTemplate.from_template("""
You are an assistant tasked with summarizing tables.
1. Provide **appropriate name** for the table that clearly reflects its content.
2. Give a concise summary of the table's key data, trends, or insights.
3. Add context to explain the table's purpose, relevance, or what it conveys.
4. At the end Provide **Alternative Name** for the table that clearly reflects its content.


Respond in the following format:
- Table Name: [Name of the table]
- Summary: [Concise summary of key data/trends]
- Context: [Explanation of the table's purpose or relevance]
- Alternative Name: [Alternative name for the table]

Do not include any additional comments or introductory phrases.

Table content: {element}
""")


summarize_chain = {"element": lambda x: x} | summary_prompt | model | StrOutputParser()

In [ ]:
import os
import json

# Define the path to the directory containing JSON files
BASE_PATH = "/content/drive/MyDrive/Final52RAG/output"



# Function to process table elements based on 'text_as_html'
def process_and_summarize_tables(all_json_data):

    for data in all_json_data:
          metadata = data.get("metadata", {})
          element_text = metadata.get("text_as_html", "")  # Identify using text_as_html
          if element_text and "<table>" in element_text:  # Check for 'table' in HTML
              summary = summarize_chain.invoke(element_text)  # Generate summary
              data["summary"] = summary  # Add summary to the element
              print(f"Processed element with table content in metadata.")

# Iterate through files in the directory
for file_name in os.listdir(BASE_PATH):
    file_path = os.path.join(BASE_PATH, file_name)

    # Process only JSON files
    if file_name.endswith(".json"):
        print(f"Processing file: {file_name}")

        # Load JSON data
        with open(file_path, "r") as json_file:
            all_json_data = json.load(json_file)


        # Process and summarize tables using 'text_as_html'
        process_and_summarize_tables(all_json_data)

        # Save the modified JSON back to the same file
        with open(file_path, "w") as json_file:
            json.dump(all_json_data, json_file, indent=4)
            print(f"Updated file saved: {file_name}")

print("Processing complete.")


Processing file: Amazon-com-Inc-2023-Annual-Report.pdf.json
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in metadata.
Processed element with table content in 

# Filtering the json

In [ ]:
import os
import json

# Define the input and output directories
INPUT_FOLDER = "/content/drive/MyDrive/Final52RAG/output"
OUTPUT_FOLDER = "/content/drive/MyDrive/Final52RAG/filtered_output"

# Ensure the output folder exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Function to extract the required fields from a JSON element
def extract_required_fields(element):
    """Extracts the specified fields from a JSON element."""
    return {
        "type": element.get("type"),
        "element_id": element.get("element_id"),
        "text": element.get("text"),
        "Summary": element.get("summary"),
        "text_as_html": element.get("metadata", {}).get("text_as_html", ""),
        "filename": element.get("metadata", {}).get("filename"),
        "page_number": element.get("metadata", {}).get("page_number"),
        "is_continuation": element.get("metadata", {}).get("is_continuation", False)
    }

# Function to process JSON files
def process_json_file(input_path, output_path):
    """Reads a JSON file, extracts required fields, and saves to a new file."""
    with open(input_path, "r") as file:
        data = json.load(file)

    extracted_data = []

    # Traverse the JSON structure, including nested elements
    for entry in data:
        if isinstance(entry, dict):
            chunks = entry.get("chunks", []) if "chunks" in entry else [entry]
            for chunk in chunks:
                extracted_fields = extract_required_fields(chunk)
                if extracted_fields["type"]:  # Process only valid types
                    extracted_data.append(extracted_fields)

    # Save the filtered data to the output file
    if extracted_data:
        with open(output_path, "w") as out_file:
            json.dump(extracted_data, out_file, indent=4)
        print(f"Saved: {output_path}")
    else:
        print(f"No valid data in: {input_path}")

# Process all JSON files in the input directory
for file_name in os.listdir(INPUT_FOLDER):
    if file_name.endswith(".json"):
        input_file_path = os.path.join(INPUT_FOLDER, file_name)
        output_file_path = os.path.join(OUTPUT_FOLDER, file_name)
        print(f"Processing: {file_name}")
        process_json_file(input_file_path, output_file_path)

print("Processing complete.")


Processing: Amazon-com-Inc-2023-Annual-Report.pdf.json
Saved: /content/drive/MyDrive/Final52RAG/filtered_output/Amazon-com-Inc-2023-Annual-Report.pdf.json
Processing: AMZN-Q1-2024-Earnings-Release.pdf.json
Saved: /content/drive/MyDrive/Final52RAG/filtered_output/AMZN-Q1-2024-Earnings-Release.pdf.json
Processing: amzn-2023-conflict-minerals-report.pdf.json
Saved: /content/drive/MyDrive/Final52RAG/filtered_output/amzn-2023-conflict-minerals-report.pdf.json
Processing: 2023-political-engagement-statement.pdf.json
Saved: /content/drive/MyDrive/Final52RAG/filtered_output/2023-political-engagement-statement.pdf.json
Processing: amazon-com-inc-bylaws-may-3-2024-1.pdf.json
Saved: /content/drive/MyDrive/Final52RAG/filtered_output/amazon-com-inc-bylaws-may-3-2024-1.pdf.json
Processing: AMZN-Q2-2024-Earnings-Release.pdf.json
Saved: /content/drive/MyDrive/Final52RAG/filtered_output/AMZN-Q2-2024-Earnings-Release.pdf.json
Processing: Amazon's-Tax-Principles.pdf.json
Saved: /content/drive/MyDrive/Fin

# Langchain Documents

In [ ]:
import os
import json
from langchain.schema import Document

# Define paths
INPUT_FOLDER = "/content/drive/MyDrive/Final52RAG/filtered_output"
OUTPUT_FOLDER = "/content/drive/MyDrive/Final52RAG/documents_output"

# Ensure the output folder exists
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# List to store LangChain documents
langchain_documents = []

# Process each filtered JSON file
for file_name in os.listdir(INPUT_FOLDER):
    if file_name.endswith(".json"):
        input_file_path = os.path.join(INPUT_FOLDER, file_name)

        # Load the JSON data
        with open(input_file_path, "r") as json_file:
            data = json.load(json_file)

        # Convert each entry into a LangChain Document
        for entry in data:
            # Extract fields
            text = entry.get("text", "")
            text_as_html = entry.get("text_as_html", "")
            summary = entry.get("Summary", "")

            # Build combined page_content, adding fields only if available
            content_parts = [text]
            if text_as_html:
                content_parts.append(f"HTML Content: {text_as_html}")
            if summary:
                content_parts.append(f"Summary: {summary}")

            combined_page_content = "\n".join(content_parts)

            # Build metadata
            metadata = {
                "element_id": entry.get("element_id", ""),
                "filename": entry.get("filename", ""),
                "page_number": entry.get("page_number", ""),
                "is_continuation": entry.get("is_continuation", False)
            }

            # Create a LangChain Document object
            document = Document(page_content=combined_page_content, metadata=metadata)
            langchain_documents.append(document)

        print(f"Processed file: {file_name}")

# Save the documents to a file for future use
output_file_path = os.path.join(OUTPUT_FOLDER, "langchain_documents.json")

# Serialize documents for saving
serialized_docs = [
    {"page_content": doc.page_content, "metadata": doc.metadata}
    for doc in langchain_documents
]

with open(output_file_path, "w") as out_file:
    json.dump(serialized_docs, out_file, indent=4)

print(f"Saved LangChain documents to: {output_file_path}")


Processed file: Amazon-com-Inc-2023-Annual-Report.pdf.json
Processed file: AMZN-Q1-2024-Earnings-Release.pdf.json
Processed file: amzn-2023-conflict-minerals-report.pdf.json
Processed file: 2023-political-engagement-statement.pdf.json
Processed file: amazon-com-inc-bylaws-may-3-2024-1.pdf.json
Processed file: AMZN-Q2-2024-Earnings-Release.pdf.json
Processed file: Amazon's-Tax-Principles.pdf.json
Processed file: amazon_officers.pdf.json
Processed file: Amazon-com-Inc-2023-Shareholder-Letter.pdf.json
Processed file: amazon_corporate_governance_guidelines.pdf.json
Processed file: Amazon-2022-Annual-Report.pdf.json
Processed file: amazon_code_of_conduct.pdf.json
Processed file: Guide to 10k.pdf.json
Processed file: Note-on-Alignment-with-Paris-Agreement.pdf.json
Processed file: AMZN-Q3-2024-Earnings-Release.pdf.json
Processed file: Amazon-Sustainable-Bond-Framework.pdf.json
Processed file: Amazon-s-Global-Economic-Impact-and-Tax-Contribution-2023.pdf.json
Processed file: Amazon-com-Inc-202